# Исследование надёжности заёмщиков

## Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** - определить влияют ли первичные параметры заемщиков на появление задолженности при выплате кредита и таким образом помочь скорректировать скоринговую модель банка. В частности необходимо проверить ряд гипотез:

 - Есть ли зависимость между наличием детей и возвратом кредита в срок?
 - Есть ли зависимость между семейным положением и возвратом кредита в срок?
 - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
 - Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**

Данные о надежности заемщиков получаются из файла data.csv. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится провести обзор данных.
Будет проведена проверка данных на ошибки и оценнено их влияние на исследование. Затем, на этапе предобработки будут исправлены самые критичные ошибки данных.

Таким образом, исследование пройдёт в три этапа:
Обзор данных.
Предобработка данных.
Проверка гипотез.

___
## Обзор данных
___

Составим первое представление о данных

Импортируем библиотеку pandas - Основной инструмент аналитика

In [95]:
import pandas as pd

Прочитаем файл data.csv из папки /datasets и сохраните его в переменной df:

In [96]:
try:
    df = pd.read_csv('/Users/vladislav/Documents/datasets/data.csv')
except:
    df = pd.read_csv('/datasets/data.csv')

Выведем на экран первые десять строк таблицы:

In [97]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Также получим общую информацию о таблице:

In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


И проведем верхнеруровневый числовой анализ данных:

In [99]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
children,21525.0,NaN,NaN,NaN,0.538908,1.381587,-1.0,0.0,0.0,1.0,20.0
days_employed,19351.0,NaN,NaN,NaN,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,401755.400475
dob_years,21525.0,NaN,NaN,NaN,43.29338,12.574584,0.0,33.0,42.0,53.0,75.0
education,21525,15,среднее,13750,NaN,NaN,NaN,NaN,NaN,NaN,NaN
education_id,21525.0,NaN,NaN,NaN,0.817236,0.548138,0.0,1.0,1.0,1.0,4.0
family_status,21525,5,женат / замужем,12380,NaN,NaN,NaN,NaN,NaN,NaN,NaN
family_status_id,21525.0,NaN,NaN,NaN,0.972544,1.420324,0.0,0.0,0.0,1.0,4.0
gender,21525,3,F,14236,NaN,NaN,NaN,NaN,NaN,NaN,NaN
income_type,21525,8,сотрудник,11119,NaN,NaN,NaN,NaN,NaN,NaN,NaN
debt,21525.0,NaN,NaN,NaN,0.080883,0.272661,0.0,0.0,0.0,0.0,1.0


Итак, в таблице **11** столбцов. Тип данных в столбцах — **int64,float64,object**.

Согласно документации к данным:
* **children** — количество детей в семье
* **days_employed** — общий трудовой стаж в днях
* **dob_years** — возраст клиента в годах
* **education** — уровень образования клиента
* **education_id** — идентификатор уровня образования
* **family_status** — семейное положение
* **family_status_id** — идентификатор семейного положения
* **gender** — пол клиента
* **income_type** — тип занятости
* **debt** — имел ли задолженность по возврату кредитов
* **total_income** — ежемесячный доход
* **purpose** — цель получения кредита

В названиях колонок нарушения стиля отсутствуют.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

В столбцах с числовыми значениями **children,days_employed,dob_years** уже на данном этапе можно заметить  артефакты - отрицательное количество дней общего стажа и число детей, "нулевой" возраст заемщиков.

**Выводы**

В каждой строке таблицы — данные о заемщике. Большая часть колонок описывает социальный портрет заемщика. Данные в столбце debt являются ключевыми для исследования - они отвечают на вопрос о кредитном качестве данного заемщика.
Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных.
Чтобы двигаться дальше, нужно их обработать.

___
## Предобработка данных
___
Исключим пропуски. Затем проверим данные на дубликаты.

### Пропуски значений
Посчитаем, сколько в таблице пропущенных значений:

In [100]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропущенные значения влияют на проверку не всех гипотез. Так в **total_income** пропуски не важны для проверки гипотез о наличии зависимости семейного положения и кредитоспособности, но важны для изучения зависимости кредитоспособности от уровня дохода. Пропуски в **days_employed** не важны для исследования.

На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. 

Придётся:

* заполнить эти пропуски явными обозначениями, 
* оценить, насколько они повредят расчётам.

Удалять строки с пропусками в столбце **days_employed** нецелесообразно, так как в данных строках имеется полезная информация о количестве детей, семейном положении и цели кредита, что позволит ответить на другие поставленные перед исследованием вопросы. Заменим пропуски в нем на 0.

In [101]:
df['days_employed'] = df['days_employed'].fillna(0)

Аналогично поступить с пропусками в столбце **total_income** нельзя, так как в дальнейшем это может повлиять на консолидированный показатель дохода по всему столбцу и, соответственно, на выводы о зависимости кредитоспособности от дохода.

Корректнее будет заменить пропуски в **total_income** на медианное значение по столбцу.

In [102]:
median_income = df['total_income'].median()
df['total_income'] = df['total_income'].fillna(median_income)

Проверим, что пропусков больше нет:

In [103]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Следующим шагом надо разобраться с отрицательными значениями в столбце **days_employed**. Изучим для какого типа занятости отображаются отрицательные значения. Сгруппируем данные по типу занятости и найдем максимальное значение трудового стажа в днях.

In [104]:
df.groupby('income_type')['days_employed'].max()

income_type
безработный        395302.838654
в декрете           -3296.759962
госслужащий             0.000000
компаньон               0.000000
пенсионер          401755.400475
предприниматель         0.000000
сотрудник               0.000000
студент              -578.751554
Name: days_employed, dtype: float64

Видно, что положительные значения характерны только для "неработающих" групп населения: "безработный" и "пенсионер". Похоже, что при выгрузке данных для работающих групп населения произошла техническая ошибка, отображающая стаж со знаком минус. Если бы стояла задача определить кредитоспособность в зависимости от стажа, необходимо было бы уточнить в источнике данных действительно ли все так. Но так как для целей исследования данное изменение не критично, можно просто получить аболютное значение из столбца **days_employed** для избавления от артефактов.

Для этого получим абсолютное значение из столца **'days_employed'** и внесем его в новый столбец **'abs_days_employed'**:

In [105]:
#Первоначальная версия кода:
#def absolute(days_employed):
#    return abs(days_employed)
#check = -10
#print(absolute(check))

#Версия кода после комментария:
df['abs_days_employed'] = df['days_employed'].abs()

Выведем верхние 5 строк для проверки:

In [106]:
#df['abs_days_employed'] = df['days_employed'].apply(absolute) #Первоначальная версия кода
display(df.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,abs_days_employed
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,8437.673028
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,4024.803754
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,5623.422610
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,4124.747207
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,340266.072047


Абсолютные значения получены. Можно удалить столбец **days_employed**:

In [107]:
del df['days_employed']
display(df.head())

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,abs_days_employed
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,8437.673028
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,4024.803754
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,5623.422610
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,4124.747207
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,340266.072047


### Дубликаты

Проверим количество возможных дубликатов в столбце **children**:

In [108]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Ввиду отсутствия возможности уточнить информацию и сопоставить ее со смысловыми значениями в других столбцах, заменим некорректные значения:

In [109]:
try:
    df['children'] = df['children'].replace(-1,1)
    df['children'] = df['children'].replace(20,2)
except:
    print('Искомое значение не найдено')
    
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Замены произведены, ошибок при замене не возникло.

Далее проверим наличие ошибок и дубликатов в столбце со значением возраста **dob_years**:

In [110]:
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

В данных по возрасту есть неверные значения: возраст части людей не указан, стоит значение "0". Заменим его на медианное значение.

In [111]:
df['dob_years'] = df['dob_years'].replace(0,df['dob_years'].median())
df['dob_years'].unique()

array([42., 36., 33., 32., 53., 27., 43., 50., 35., 41., 40., 65., 54.,
       56., 26., 48., 24., 21., 57., 67., 28., 63., 62., 47., 34., 68.,
       25., 31., 30., 20., 49., 37., 45., 61., 64., 44., 52., 46., 23.,
       38., 39., 51., 59., 29., 60., 55., 58., 71., 22., 73., 66., 69.,
       19., 72., 70., 74., 75.])

Замена произведена корректно. Теперь проверим данные по полу заемщиков в столбце **gender**:

In [112]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Как видно, имеется артефакт - "XNA". Так как женщин в выборке в 2 раза больше, вероятность того, что неуказанный пол заемщика также был женским выше. Внесем это значение.

In [113]:
df['gender'] = df['gender'].replace('XNA','F')
df['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

Замена произведена корректно.

Далее проверим количество возможных дубликатов в столбце **family_status**:

In [114]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

В данном столбце дубликатов нет. Также в таблице мы видим столбец **family_status_id**. Сопоставим его значения со столбцом **family_status**

In [115]:
df.groupby('family_status')['family_status_id'].mean()

family_status
Не женат / не замужем    4
в разводе                3
вдовец / вдова           2
гражданский брак         1
женат / замужем          0
Name: family_status_id, dtype: int64

Мы видим, что каждому конкретному статусу соответствует определенный id. Для того, чтобы облегчить датафрейм можно вынести избыточные данные в отдельную таблицу и удалить из нее дубликаты.

In [116]:
family_status_dict = df[['family_status','family_status_id']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
print(family_status_dict)

           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  Не женат / не замужем                 4


Можно удалить столбец **family_status** из основной таблицы df:

In [117]:
del df['family_status']
display(df.head())

,children,dob_years,education,education_id,family_status_id,gender,income_type,debt,total_income,purpose,abs_days_employed
0,1,42.0,высшее,0,0,F,сотрудник,0,253875.639453,покупка жилья,8437.673028
1,1,36.0,среднее,1,0,F,сотрудник,0,112080.014102,приобретение автомобиля,4024.803754
2,0,33.0,Среднее,1,0,M,сотрудник,0,145885.952297,покупка жилья,5623.422610
3,3,32.0,среднее,1,0,M,сотрудник,0,267628.550329,дополнительное образование,4124.747207
4,0,53.0,среднее,1,1,F,пенсионер,0,158616.077870,сыграть свадьбу,340266.072047


Аналогичным образом следует поступить со столбцами об образовании, при условии, что в данных нет разночтений и определенному значению по образованию соответствует строго определенный id. Для этого выведем среднее id для каждого значения, так как малейшее отколенение от целого числа покажет наличие разных id для одного и того уровня образования:

In [118]:
df.groupby('education')['education_id'].mean()

education
ВЫСШЕЕ                 0
Высшее                 0
НАЧАЛЬНОЕ              3
НЕОКОНЧЕННОЕ ВЫСШЕЕ    2
Начальное              3
Неоконченное высшее    2
СРЕДНЕЕ                1
Среднее                1
УЧЕНАЯ СТЕПЕНЬ         4
Ученая степень         4
высшее                 0
начальное              3
неоконченное высшее    2
среднее                1
ученая степень         4
Name: education_id, dtype: int64

Все числа целые, значит конкретному уровню образования соответствует конкретный id. При этом имеются скрытые дубликаты, от которых можно будет избавиться в отдельной таблице:

In [119]:
pd.options.mode.chained_assignment = None
education_dict = df[['education','education_id']]
education_dict['education_lowercase'] = education_dict['education'].str.lower()
education_dict = education_dict.groupby('education_lowercase')['education_id'].mean().sort_values()
display(education_dict)

education_lowercase
высшее                 0
среднее                1
неоконченное высшее    2
начальное              3
ученая степень         4
Name: education_id, dtype: int64

Можно удалить столбец **education** из основной таблицы df:

In [120]:
del df['education']
display(df.head())

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,abs_days_employed
0,1,42.0,0,0,F,сотрудник,0,253875.639453,покупка жилья,8437.673028
1,1,36.0,1,0,F,сотрудник,0,112080.014102,приобретение автомобиля,4024.803754
2,0,33.0,1,0,M,сотрудник,0,145885.952297,покупка жилья,5623.422610
3,3,32.0,1,0,M,сотрудник,0,267628.550329,дополнительное образование,4124.747207
4,0,53.0,1,1,F,пенсионер,0,158616.077870,сыграть свадьбу,340266.072047


Завершающм этапом обработки ошибок посчитаем явные дубликаты:

In [121]:
print("Количество явных дубликатов: {}".format(df.duplicated().sum()))

Количество явных дубликатов: 72


Удалим явные дубликаты:

In [122]:
df = df.drop_duplicates().reset_index(drop=True)

Проверим, что явные дубликаты удалены:

In [123]:
#print("Количество явных дубликатов: {}".format(df.duplicated().sum()))
print(f"Количество явных дубликатов: {df.duplicated().sum()}")

Количество явных дубликатов: 0


### Категоризация

Для ответа на вопрос о зависимости наличия детей и возвратом кредита в срок необходимо добавить столбец в котором будет отражена информация о самом факте наличия детей, а не об их количестве. Для этого напишем соответствующую функцию и применим ее:

In [124]:
def has_children(children):
    if children >= 1:
        return 'есть дети'
    else:
        return 'детей нет'

In [125]:
df['has_children'] = df['children'].apply(has_children)
df.head(10)

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,abs_days_employed,has_children
0,1,42.0,0,0,F,сотрудник,0,253875.639453,покупка жилья,8437.673028,есть дети
1,1,36.0,1,0,F,сотрудник,0,112080.014102,приобретение автомобиля,4024.803754,есть дети
2,0,33.0,1,0,M,сотрудник,0,145885.952297,покупка жилья,5623.422610,детей нет
3,3,32.0,1,0,M,сотрудник,0,267628.550329,дополнительное образование,4124.747207,есть дети
4,0,53.0,1,1,F,пенсионер,0,158616.077870,сыграть свадьбу,340266.072047,детей нет
5,0,27.0,0,1,M,компаньон,0,255763.565419,покупка жилья,926.185831,детей нет
6,0,43.0,0,0,F,компаньон,0,240525.971920,операции с жильем,2879.202052,детей нет
7,0,50.0,1,0,M,сотрудник,0,135823.934197,образование,152.779569,детей нет
8,2,35.0,0,1,F,сотрудник,0,95856.832424,на проведение свадьбы,6929.865299,есть дети
9,0,41.0,1,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,2188.756445,детей нет


Функция отработала корректно.

Далее изучим столбец **total_income**. В столбце представленны данные в формате вещественных чисел. Для более точных расчетов в последующем нам потребуется перевести данные в формат целых чисел.

In [126]:
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   children           21453 non-null  int64  
 1   dob_years          21453 non-null  float64
 2   education_id       21453 non-null  int64  
 3   family_status_id   21453 non-null  int64  
 4   gender             21453 non-null  object 
 5   income_type        21453 non-null  object 
 6   debt               21453 non-null  int64  
 7   total_income       21453 non-null  int64  
 8   purpose            21453 non-null  object 
 9   abs_days_employed  21453 non-null  float64
 10  has_children       21453 non-null  object 
dtypes: float64(2), int64(5), object(4)
memory usage: 1.8+ MB


Видим, что формат успешно изменен. 

Для более удобного анализа данных ежемесячный доход лучше разбить на категории. Чтобы определить оптимальный диапазон категорий изучим диапазон значений в столбце **total_income**:

In [127]:
display(df['total_income'].min())
display(df['total_income'].max())
display(df['total_income'].mean())
display(df['total_income'].median())

20667

2265604

165226.54262807066

145017.0

Видно, что минимальное значение составляет *20667*, а максимальное *2265604*, при этом среднее и медиана находятся в диапазоне *145017 - 165209*. Можно предположить, что доход большей части людей находится в диапазоне *100000-200000*. Заложим дополнительно ±*50000* от верхней и нижней предполагаемой границы для более широкого охвата и напишем функцию: 

In [128]:
def income_range(total_income):
    if total_income < 50000:
        return '< 50k'
    elif total_income <= 100000:
        return '50-100k'
    elif total_income <= 150000:
        return '100-150k'
    elif total_income <=200000:
        return '150-200k'
    elif total_income <=250000:
        return '200-250k'
    else:
        return '>250k'
check = 159000
print(income_range(check))

150-200k


Функция отработала верно. Теперь добавим новый столбец значения в который вернет данная функция.

In [129]:
df['total_income_range'] = df['total_income'].apply(income_range)
df.head()

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,abs_days_employed,has_children,total_income_range
0,1,42.0,0,0,F,сотрудник,0,253875,покупка жилья,8437.673028,есть дети,>250k
1,1,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,4024.803754,есть дети,100-150k
2,0,33.0,1,0,M,сотрудник,0,145885,покупка жилья,5623.422610,детей нет,100-150k
3,3,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,4124.747207,есть дети,>250k
4,0,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,340266.072047,детей нет,150-200k


Сгруппируем данные по столбцу **total_income_range** и выделим долю каждого диапазона, чтобы убедиться, что категории установлены равномерно:

In [130]:
total_income_range_df = df.groupby('total_income_range').agg({'debt':'count'})
total_income_range_df['percentage'] = (total_income_range_df['debt']/df['debt'].count())*100
total_income_range_df

,debt,percentage
total_income_range,,
100-150k,7806,36.386519
150-200k,4118,19.195451
200-250k,2254,10.506689
50-100k,4091,19.069594
< 50k,372,1.734023
>250k,2812,13.107724


Отсортируем категории по мере возрастания дохода для наглядности:

In [131]:
total_income_range_df['order'] = [3,4,5,2,1,6]
total_income_range_df = total_income_range_df.sort_values(by='order')
del total_income_range_df['order']
total_income_range_df

,debt,percentage
total_income_range,,
< 50k,372,1.734023
50-100k,4091,19.069594
100-150k,7806,36.386519
150-200k,4118,19.195451
200-250k,2254,10.506689
>250k,2812,13.107724


Видно, что данные распределены по категориям без сильных перекосов в крайних диапазонах, значит категории определены верно. 

**При этом следует отметить, что доля заемщиков с уровнем дохода менее 50000 составляет менее 2% от всей выборки, что не позволяет делать выводы по данной группе ввиду недостаточности данных**.

### Лемматизация

Следующим этапом проанализируем данные по цели получения кредита в столбце **purpose**. Проанализируем данные:

In [132]:
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     767
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Из полученных данных видно, что наиболее часто упоминаемые цели получения кредита: 

* **приобретение недвижимости**,
* **покупка автомобиль**,
* **получение образования**,
* **проведение свадьбы**.

Также видно, что в столбце много скрытых дубликатов из-за разной формулировки заемщиками цели получаемого кредита. Необходимо привести данный список к единообразию формулировок, для этого используем лемматизацию:

In [133]:
from pymystem3 import Mystem
m = Mystem()

lemma_string = str(df['purpose'])
lemma = m.lemmatize(lemma_string)

Для более удобного применения лемматизации напишем функцию:

In [134]:
def get_purpose_group(purpose):
    lemma = m.lemmatize(purpose)
    if ('недвижимость' in lemma) or ('жилье' in lemma) or ('строительство' in lemma):
        return 1
    elif 'автомобиль' in lemma:
        return 2
    elif 'образование' in lemma:
        return 3
    elif 'свадьба' in lemma:
        return 4
    else:
        return 5

Создадим в таблице новый столбец, который с помощью написанной функции присвоит категорию и выведем таблицу на экран для проверки:

In [135]:
df['purpose_id'] = df['purpose'].apply(get_purpose_group)
df.head()

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,abs_days_employed,has_children,total_income_range,purpose_id
0,1,42.0,0,0,F,сотрудник,0,253875,покупка жилья,8437.673028,есть дети,>250k,1
1,1,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,4024.803754,есть дети,100-150k,2
2,0,33.0,1,0,M,сотрудник,0,145885,покупка жилья,5623.422610,детей нет,100-150k,1
3,3,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,4124.747207,есть дети,>250k,3
4,0,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,340266.072047,детей нет,150-200k,4


Создадим таблицу с категориями для разных целей получения кредита:

In [136]:
columns_title = ['purpose','purpose_id']
purpose_data=[['приобретение недвижимости',1],
              ['покупка автомобиля',2],
              ['получение образования',3],
              ['проведение свадьбы',4],
              ['иное',5]
             ]
purpose_dict = pd.DataFrame(data=purpose_data, columns=columns_title)
purpose_dict

,purpose,purpose_id
0,приобретение недвижимости,1
1,покупка автомобиля,2
2,получение образования,3
3,проведение свадьбы,4
4,иное,5


Теперь можно удалить столбец **purpose** из общей таблицы:

In [137]:
df = df.drop('purpose', axis=1)
df.head()

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,abs_days_employed,has_children,total_income_range,purpose_id
0,1,42.0,0,0,F,сотрудник,0,253875,8437.673028,есть дети,>250k,1
1,1,36.0,1,0,F,сотрудник,0,112080,4024.803754,есть дети,100-150k,2
2,0,33.0,1,0,M,сотрудник,0,145885,5623.422610,детей нет,100-150k,1
3,3,32.0,1,0,M,сотрудник,0,267628,4124.747207,есть дети,>250k,3
4,0,53.0,1,1,F,пенсионер,0,158616,340266.072047,детей нет,150-200k,4


In [138]:
#del df['purpose']
#df.head()

Данным действием можно закончить этап предобработки данных.

По итогу предобработки данных получен датафрейм с базой заемщиков включающий информацию по ряду ключевых для анализа показателей:

   * наличие детей
   * семейное положение (в формате индексов категорий)
   * уровень дохода (в формате диапазонов категорий)
   * цели кредита (в формате индексов категорий)

Также были созданы вспомогательные таблицы:
    
   * family_status_dict (с индексами категорий о семейном положении)
   * purpose_dict (с индексами категорий о цели кредита)

Подготовленные данные помогут ответить на поставленные вопросы.

___
## Проверка гипотез
___

Проверим наличие зависимости от наличия детей и возвратом кредита в срок. Для этого построим сводную таблицу по интересующим нас данным:

In [140]:
df_pivot_children = df.pivot_table(index='has_children',values='debt',aggfunc='mean')
df_pivot_children['debt'] = pd.Series(["{0:.2f}%".format(val * 100) for val in df_pivot_children['debt']], index = df_pivot_children.index)
df_pivot_children

,debt
has_children,
детей нет,7.54%
есть дети,9.21%


По итогам данного сравнения можно увидеть, что среди заемщиков с детьми доля имевших задолженность по возврату кредитов выше, чем среди заемщиков без детей примерно на 1,67%.
Таким образом, можно утверджать, что 

**зависимость между наличием детей и возвратом кредита в срок имеется**.
* * *

Проверим наличие зависимости между семейным положением и возвратом кредита в срок. Для этого также построим сводную таблицу по соответствующим данным:

In [45]:
df_pivot_family = df.pivot_table(index='family_status_id',values='debt',aggfunc='mean')
df_pivot_family['debt'] = pd.Series(["{0:.2f}%".format(val * 100) for val in df_pivot_family['debt']], index = df_pivot_family.index)
df_pivot_family

,debt
family_status_id,
0,7.55%
1,9.35%
2,6.57%
3,7.11%
4,9.75%


Далее присоединим данную таблицу к таблице с обозначением индексов категорий для понимания статуса каждой группы:

In [46]:
family_status_dict.merge(df_pivot_family,on='family_status_id',how='right')

,family_status,family_status_id,debt
0,женат / замужем,0,7.55%
1,гражданский брак,1,9.35%
2,вдовец / вдова,2,6.57%
3,в разводе,3,7.11%
4,Не женат / не замужем,4,9.75%


По итогам данного сравнения можно увидеть, что средние значения по наличию задолженности по возврату кредитов между группами заемщиков с разным семейным положением варьируется в диапазоне от 6,57% до 9,75%. Наиболее низкое значение по наличию задолженности характерно для людей со статусом "вдовец/вдова", у заемщиков в статусе "не женат/не замужем" отмечен наиболее высокий показатель. Таким образом, можно утверджать, что

**зависимость между семейным положением и возвратом кредита в срок имеется**.
* * *

Далее проверим наличие зависимости между уровнем дохода и возвратом кредита в срок. Для этого также построим сводную таблицу по соответствующим данным:

In [47]:
df_pivot_income = df.pivot_table(index='total_income_range',values='debt',aggfunc='mean')
df_pivot_income['debt'] = pd.Series(["{0:.2f}%".format(val * 100) for val in df_pivot_income['debt']], index = df_pivot_income.index)
df_pivot_income

,debt
total_income_range,
100-150k,8.47%
150-200k,8.94%
200-250k,7.28%
50-100k,8.09%
< 50k,6.18%
>250k,6.90%


Отсортируем таблицу по возрастанию уровня дохода для большей наглядности:

In [48]:
df_pivot_income['order'] = [3,4,5,2,1,6]
df_pivot_income = df_pivot_income.sort_values(by='order')
del df_pivot_income['order']
df_pivot_income

,debt
total_income_range,
< 50k,6.18%
50-100k,8.09%
100-150k,8.47%
150-200k,8.94%
200-250k,7.28%
>250k,6.90%


По итогам данного сравнения можно увидеть, что наименьшая доля заемщиков имевших задолженность по возврату кредита 
характерна для заемщиков с уровнем минимальным и максимальным уровнем дохода (до 50000 и от 200000 и выше). 
В то же время заемщики со средним по выборке уровнем дохода 50000-200000 имели наиболее высокие значения по наличию
задолженности. 

При этом мы помним, что доля заемщиков с доходом ниже 50000 составляет менее 2% от всей выборки, что
не является репрезентативным значением для утверждения о наличии зависимости по данной группе.

По итогу можно утверджать, что наиболее низкие показатели по наличию задолженности характерны для заемщиков с 
высоким уровнем дохода - от 200000 и выше. Таким образом, действительно утверждение, что

**зависимость между уровнем дохода и возвратом кредита в срок имеется**.
* * *

Также проверим как разные цели кредита влияют на возврат его в срок. Для этого также построим сводную таблицу по соответствующим данным:

In [49]:
df_pivot_purpose = df.pivot_table(index='purpose_id',values='debt',aggfunc='mean')
df_pivot_purpose['debt'] = pd.Series(["{0:.2f}%".format(val * 100) for val in df_pivot_purpose['debt']], index = df_pivot_purpose.index)
df_pivot_purpose

,debt
purpose_id,
1,7.23%
2,9.36%
3,9.22%
4,8.01%


Далее присоединим данную таблицу к таблице с обозначением индексов категорий для понимания целей каждой группы:

In [50]:
purpose_dict.merge(df_pivot_purpose,on='purpose_id',how='right')

,purpose,purpose_id,debt
0,приобретение недвижимости,1,7.23%
1,покупка автомобиля,2,9.36%
2,получение образования,3,9.22%
3,проведение свадьбы,4,8.01%


По итогам данного сравнения можно увидеть, что средние значения по наличию задолженности по возврату кредитов между группами заемщиков с разными целями получени кредита варьируется в диапазоне от 7,23% до 9,35%. Наиболее низкое значение по наличию задолженности характерно для людей берущих кредит на "приобретение недвижимости", у заемщиков берущих кредит на "покупку автомобиля" отмечен наиболее высокий показатель. Таким образом, можно утверджать, что

**цель получения кредита влияет на его выплату в срок**.
* * *

**Выводы**

Все гипотезы подтвердились:
* зависимость между наличием детей и возвратом кредита в срок имеется
* зависимость между семейным положением и возвратом кредита в срок имеется
* зависимость между уровнем дохода и возвратом кредита в срок имеется
* цель получения кредита влияет на его выплату в срок
_ _ _ 

## Итоги исследования

Были проверены четыре гипотезы и установлено:
* Заемщики без детей реже допускали появление задолженности по выплате кредита.
* Заемщики имеющие статус семейного положения "гражданский брак" и "не женат/не замужем" чаще всего имели 
задолженность по выплате кредита в срок.
* Уровень дохода заемщиков и возврат кредита в срок также имеют зависимость, но данная гипотеза подтвердилась отчасти
ввиду небольшого числа заемщиков с низким уровнем дохода. Люди с высоким уровнем дохода (от 200000) реже допускали
появление задолженности по выплате кредита в срок.
* Цель получения кредита влияет на его выплату в срок: люди бравшие кредит на покупку недвижимости и проведение 
свадьбы чаще отдавали кредит в срок, в то время как для людей с целью получения кредита "покупка автомобиля" и 
"получение образования" характерен более высокий показатель по наличию задолженности при возврате кредита.